In [1]:
import pandas as pd 
import numpy as np 
import torch
import transformers
from tqdm import tqdm 

from torch.utils.data import Dataset, DataLoader
from transformers import DebertaTokenizer, DebertaModel
import logging
logging.basicConfig(level=logging.ERROR)
from tqdm import tqdm
import matplotlib.pyplot as plt
from IPython.display import clear_output

import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
dataset = pd.read_csv('../data/updated_test.csv') 
unique_labels =dataset.updated_label.unique().tolist() 
print("Unique Labels in the Dataset:\n\n ", unique_labels)

Unique Labels in the Dataset:

  ['fallacy of extension', 'faulty generalization', 'fallacy of logic', 'false causality', 'fallacy of credibility', 'circular reasoning', 'ad hominem', 'ad populum', 'intentional', 'fallacy of relevance', 'equivocation', 'false dilemma']


In [2]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [3]:
import nvidia_smi
nvidia_smi.nvmlInit()

handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
# card id 0 hardcoded here, there is also a call to get all available card ids, so we could iterate

info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)

print("Total memory:", info.total)
print("Free memory:", info.free)
print("Used memory:", info.used)

nvidia_smi.nvmlShutdown()

NVMLError_GpuIsLost: GPU is lost

In [5]:
print("Get memory info", torch.cuda.mem_get_info(device=None)) 
print("Get number of devices available: ", torch.cuda.device_count())

print("Memory stats about which device is free: ")

nvidia_smi.nvmlInit()

deviceCount = nvidia_smi.nvmlDeviceGetCount()
for i in range(deviceCount):
    handle = nvidia_smi.nvmlDeviceGetHandleByIndex(i)
    info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
    print("Device {}: {}, Memory : ({:.2f}% free): {}(total), {} (free), {} (used)".format(i, nvidia_smi.nvmlDeviceGetName(handle), 100*info.free/info.total, info.total, info.free, info.used))

nvidia_smi.nvmlShutdown()

print("Current Device:  ", torch.cuda.current_device()) 


Get memory info (49782652928, 50962169856)
Get number of devices available:  4
Memory stats about which device is free: 
Device 0: b'Quadro RTX 8000', Memory : (97.69% free): 50962169856(total), 49782652928 (free), 1179516928 (used)
Device 1: b'Quadro RTX 8000', Memory : (57.85% free): 50962169856(total), 29482221568 (free), 21479948288 (used)
Device 2: b'Quadro RTX 8000', Memory : (99.99% free): 50962169856(total), 50958106624 (free), 4063232 (used)
Device 3: b'Quadro RTX 8000', Memory : (14.39% free): 50962169856(total), 7332102144 (free), 43630067712 (used)
Current Device:   0


In [7]:
torch.cuda.set_device(2)
torch.cuda.empty_cache() 

In [8]:

MAX_LEN = 256
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
LEARNING_RATE = 1e-05
tokenizer = DebertaTokenizer.from_pretrained("microsoft/deberta-base")

In [9]:
class LogicalFallacy(Dataset):
    def __init__(self, dataset, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataset
        self.text = dataset.source_article
        self.targets = dataset.label
        self.max_len = max_len
        self.original_label = dataset.updated_label

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        

        return {
            'sentence': text,
            
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [10]:
class DebertaClass(torch.nn.Module):
    def __init__(self):
        super(DebertaClass, self).__init__()
        self.l1 = DebertaModel.from_pretrained("microsoft/deberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output



In [11]:

def train_loop( train_loader, test_loader, label, test_data, epochs=5):
  train_loss = []
  test_loss = []
  train_accuracy = []
  test_accuracy = []
  model = DebertaClass()
  model.to(device)
  loss_function = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

  test_answers = [[[],[]], [[],[]]]
  for epoch in range(epochs):
    for phase in ['Train', 'Test']:
      if(phase == 'Train'):
        model.train()
        loader = train_loader
      else:
        model.eval()
        loader = test_loader  
      epoch_loss = 0
      epoch_acc = 0
      len(loader)
      for steps, data in tqdm(enumerate(loader, 0)):
        sentence = data['sentence']
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)
      
        outputs = model.forward(ids, mask, token_type_ids)

        loss = loss_function(outputs, targets)        
        
        epoch_loss += loss.detach()
        _, max_indices = torch.max(outputs.data, dim=1)
        bath_acc = (max_indices==targets).sum().item()/targets.size(0)
        epoch_acc += bath_acc

        if (phase == 'Train'):
          train_loss.append(loss.detach()) 
          train_accuracy.append(bath_acc)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
        else:
          test_loss.append(loss.detach()) 
          test_accuracy.append(bath_acc)
          if epoch == epochs-1:
            for i in range(len(targets)):
              test_answers[targets[i].item()][max_indices[i].item()].append([
                                                                  sentence[i], 
                                                                  targets[i].item(), 
                                                                  max_indices[i].item()])

      print(f"{phase} Loss: {epoch_loss/steps}")
      print(f"{phase} Accuracy: {epoch_acc/steps}")
  
  torch.save(model, '../models/'+label+'_trained_deberta.pt')
  return (train_loss, test_loss, train_accuracy, test_accuracy, test_answers)


In [15]:
def driver_code(label):
    train_df = pd.read_csv('../data/'+label+'_train.csv')
    test_df = pd.read_csv('../data/'+label+'_test.csv')

    train_set = LogicalFallacy(train_df, tokenizer, MAX_LEN)
    test_set = LogicalFallacy(test_df, tokenizer, MAX_LEN)

    train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

    test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }
    train_loader = DataLoader(train_set, **train_params)
    test_loader = DataLoader(test_set, **test_params)

    
    value = train_loop( train_loader, test_loader, label,  test_df['updated_label'], epochs=10)
    return value
    

In [16]:
train_loss_list, test_loss_list, train_accuracy_list, test_accuracy_list, test_answers_list = [], [], [], [], []
for i in range(len(unique_labels)):
    print("Training model for label: ", unique_labels[i], "\n") 
    train_loss, test_loss, train_accuracy, test_accuracy, test_answers = driver_code(unique_labels[i])
    train_loss_list.append(train_loss) 
    test_loss_list.append(test_loss)
    train_accuracy_list.append(train_accuracy)
    test_accuracy_list.append(test_accuracy)
    test_answers_list.append(test_answers) 



Training model for label:  fallacy of extension 



Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaModel: ['lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.bias']
- This IS expected if you are initializing DebertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
0it [00:00, ?it/s]/nas/home/vprasann/anaconda3/envs/neuros/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2263: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use 

Train Loss: 0.7166959643363953
Train Accuracy: 0.5336538461538461


11it [00:00, 15.28it/s]
0it [00:00, ?it/s]

Test Loss: 0.6983790397644043
Test Accuracy: 0.75


...

Test Loss: 0.6121950745582581
Test Accuracy: 1.0


28it [00:05,  4.74it/s]
2it [00:00, 15.85it/s]

Train Loss: 0.011894547380506992
Train Accuracy: 1.037037037037037


6it [00:00, 14.52it/s]


Test Loss: 0.4380706250667572
Test Accuracy: 1.05


In [14]:
import pickle 
file = open('../auxillary data/test_answers_5.pk', 'wb')

# dump information to that file
pickle.dump(test_answers_list, file)

# close the file
file.close()